# Programming GPUs in Python3 using PyCuda
**[Big Data and Cloud Computing]**

Most machines have available GPUs which we should take advantage of. GPUs can greatly accelerate execution if well programmed. Here we go through some examples.

The example given in this notebook will only take advantage of a GPU if you have cuda installed and if you have a graphics card that is cuda-enabled. Please check [this wikipedia site](https://en.wikipedia.org/wiki/CUDA) or the official [nvidia site] (https://developer.nvidia.com/cuda-gpus) for more detail about compatibility. If you run this notebool in colab, it already provides you with a GPU in its runtime environment.

__References__:

- [Tutorial on PyCuda](https://documen.tician.de/pycuda/tutorial.html)
- [CUDA programming guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=b0dc11148f5d5794b3e5fe6a12a191db136dd6709a70a379f9c8e0de30cdf5d3
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


Let's start with a very simple code. We will create two vectors (A and B) and add them producing a third vector (C). The kernel code is reproduced below: (not executable yet)

In [4]:
// Kernel definition
__global__ void VecAdd(float* A, float* B, float* C)
{
    int i = threadIdx.x;
    C[i] = A[i] + B[i];
}


SyntaxError: invalid syntax (<ipython-input-4-ebfc4123d45d>, line 1)

Let's make it executable by integrating it in a Python code. The array is created using `numpy`. We conveniently use numpy when working with GPUs, because GPUs can work very well on number crunching operations.

(**NOTE**: if running in colab, you may need to configure your runtime to have a GPU. Click on the "Runtime" menu and then "Change Runtime type".)

In [3]:
############################
# GPU version
############################

# need to import modules
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
# end
import numpy
from time import time

# begin timing
start_time = time()
########
N = 16
numpy.random.seed(123)
a = numpy.random.rand(N)
a = a.astype(numpy.float32)
b = numpy.random.rand(N)
b = b.astype(numpy.float32)
c = numpy.zeros(shape=(1,N),dtype=numpy.float32)

# need to allocate memory in the GPU to fit our array
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)
# a_tid = cuda.mem_alloc(b.nbytes)
# a_bid = cuda.mem_alloc(c.nbytes)
# need to copy our array to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)
# cuda.memcpy_htod(a_tid, b)
# cuda.memcpy_htod(a_bid, c)

# here it is the kernel that will run the addition in the GPU
mod = SourceModule("""
    __global__ void VecAdd(float* A, float* B, float* C)
    {
        int i = threadIdx.x;
        C[i] = A[i] + B[i];
    }
 """)
func = mod.get_function("VecAdd")
# now, we define the shape
func(a_gpu, b_gpu, c_gpu, block=(N,1,1))

# create space in the host memory to receive results
#c = numpy.empty_like(c)
#c_tid_result = numpy.empty_like(c)

# copy results from the GPU memory to the host memory
cuda.memcpy_dtoh(c, c_gpu)

# end timing
print(round(time() - start_time,8), 'seconds')
########

# print("tid  bid\n")
# for i in range(len(b)):
#   s = ""
#   for j in range(len(b[i])):
#      s += str(b[i,j])+"  "+str(c[i,j])+"  "
#   print(s)
print(c)
print(a)
print(b)
(a+b==c)


0.98586082 seconds
[[0.8789609  0.4615911  0.7584028  1.0831423  1.3538699  1.2725383
  1.7052195  1.2958531  1.2033753  0.71507645 0.70496666 0.95731294
  0.73228633 0.69065404 0.4901492  1.1716965 ]]
[0.6964692  0.28613934 0.22685145 0.5513148  0.71946895 0.42310646
 0.9807642  0.6848297  0.4809319  0.39211753 0.343178   0.7290497
 0.43857226 0.0596779  0.39804426 0.7379954 ]
[0.18249173 0.17545176 0.53155136 0.53182757 0.63440096 0.8494318
 0.7244553  0.6110235  0.7224434  0.32295892 0.36178866 0.22826323
 0.29371405 0.63097614 0.09210494 0.4337012 ]


array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True]])

Now, let's play with a bidimensional array with dimensions RxC. In this example, we create the matrix using `numpy` again. The function simply multiplies each cell of the matrix by 2. We create first a sequential version and next a cuda version.

In [5]:
############################
# Sequential version
############################

import numpy
from time import time

R = 16
C = 16
# begin timing
start_time = time()
########

numpy.random.seed(123)
a = numpy.random.randn(R,C)
a_doubled = a*2

# end timing
print(round(time() - start_time,8), 'seconds')
########

print(a_doubled)
print(a)
a*2==a_doubled

0.00130296 seconds
[[-2.17126121e+00  1.99469089e+00  5.65956996e-01 -3.01258943e+00
  -1.15720050e+00  3.30287307e+00 -4.85335849e+00 -8.57825258e-01
   2.53187252e+00 -1.73348080e+00 -1.35777230e+00 -1.89417938e-01
   2.98277925e+00 -1.27780399e+00 -8.87963919e-01 -8.68702551e-01]
 [ 4.41186017e+00  4.37357218e+00  2.00810780e+00  7.72372798e-01
   1.47473715e+00  2.98146406e+00 -1.87166774e+00  2.35165809e+00
  -2.50776134e+00 -1.27550300e+00  1.81421039e+00 -2.85736140e+00
  -2.80137440e-01 -1.72350979e+00 -5.11238741e-01 -5.59717821e+00]
 [-3.54306621e+00 -1.39975447e+00  1.85492486e+00 -3.47271366e-01
   5.69183179e-03  1.37644542e+00 -1.75907269e+00  5.67254648e-01
  -1.61073304e+00 -3.45533899e+00 -7.81799588e-01  1.14761172e+00
   6.77178102e-01 -2.36609890e-02  4.78473053e+00  8.25824321e-01]
 [ 1.95747201e+00  4.47628668e+00 -2.58817065e+00 -2.07757642e+00
   3.48742445e+00 -1.59612547e+00  5.93664606e-02  2.13863194e+00
   1.78141278e+00  3.50977236e+00  2.99128827e+00  2.1

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [12]:
############################
# Sequential version
############################

import numpy
from time import time

R = 10240
C = 10240
# begin timing
start_time = time()
########

numpy.random.seed(123)
a = numpy.random.randn(R,C)
a_doubled = a*2

# end timing
print(round(time() - start_time,8), 'seconds')
########

print(a_doubled)
print(a)
a*2==a_doubled

3.28372169 seconds
[[-2.17126121  1.99469089  0.565957   ...  1.72446443  3.60881851
  -2.88587644]
 [-0.47114058 -1.34853623 -1.65642254 ... -3.33163526  0.38023076
   0.42766382]
 [-0.95525402  1.9486294   1.80930121 ... -2.06418585 -0.95221937
  -0.94392159]
 ...
 [-1.29066852  3.17536121  3.46727044 ...  1.32611824 -2.34483286
   0.11799299]
 [-1.82653166 -2.42549176  0.23146129 ...  2.57736052 -1.27727782
   1.14314436]
 [-2.74400789  0.57926116 -2.67865638 ...  0.09464489 -0.57723126
   0.28486693]]
[[-1.0856306   0.99734545  0.2829785  ...  0.86223221  1.80440926
  -1.44293822]
 [-0.23557029 -0.67426812 -0.82821127 ... -1.66581763  0.19011538
   0.21383191]
 [-0.47762701  0.9743147   0.9046506  ... -1.03209292 -0.47610968
  -0.4719608 ]
 ...
 [-0.64533426  1.5876806   1.73363522 ...  0.66305912 -1.17241643
   0.05899649]
 [-0.91326583 -1.21274588  0.11573065 ...  1.28868026 -0.63863891
   0.57157218]
 [-1.37200394  0.28963058 -1.33932819 ...  0.04732245 -0.28861563
   0.14243346

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

Now, let's "decorate" this code to use a GPU to compute all multiplications in parallel. In this example, we use auxiliary arrays that will contain the thread IDs and block IDs, so that we can inspect the number of threads working and what each one is doing. Thread IDs Block IDs are stored in the GPU, in vectors called a_tid and a_bid, respectively. In the host they are called b and c. Our matrix is called `a` in the host and `a_gpu` in the GPU.

In [6]:
############################
# GPU version
############################

# need to import modules
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
# end
import numpy
from time import time

# begin timing
start_time = time()
########
R = 16
C = 16
numpy.random.seed(123)
a = numpy.random.randn(R,C)
a = a.astype(numpy.float32)
b = numpy.zeros(shape=(R,C),dtype=numpy.uint32)
c = numpy.zeros(shape=(R,C),dtype=numpy.uint32)

# need to allocate memory in the GPU to fit our array
a_gpu = cuda.mem_alloc(a.nbytes)
a_tid = cuda.mem_alloc(b.nbytes)
a_bid = cuda.mem_alloc(c.nbytes)
# need to copy our array to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(a_tid, b)
cuda.memcpy_htod(a_bid, c)

# here it is the kernel that will run the multiplication in the GPU
mod = SourceModule("""
  __global__ void doublify(float *a, uint *a_tid, uint *a_bid)
  {
    int idx = threadIdx.x + threadIdx.y*16;
    a[idx] *= 2;
    a_tid[idx] = threadIdx.x + blockDim.x * threadIdx.y;
    a_bid[idx] = blockDim.x * blockDim.y;

  }
  """)
func = mod.get_function("doublify")
# now, we define the shape
func(a_gpu, a_tid, a_bid, block=(1,R,C))

# create space in the host memory to receive results
a_doubled = numpy.empty_like(a)
a_tid_result = numpy.empty_like(b)

# copy results from the GPU memory to the host memory
cuda.memcpy_dtoh(a_doubled, a_gpu)
cuda.memcpy_dtoh(b, a_tid)
cuda.memcpy_dtoh(c, a_bid)


# end timing
print(round(time() - start_time,8), 'seconds')
########

print("tid  bid\n")
for i in range(len(b)):
  s = ""
  for j in range(len(b[i])):
     s += str(b[i,j])+"  "+str(c[i,j])+"  "
  print(s)
print(a_doubled)
print(a)
(a*2==a_doubled)



/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


0.64592648 seconds
tid  bid

0  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
1  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
2  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
3  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
4  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
5  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
6  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
7  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
8  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
9  16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, Fals

In [10]:
############################
# GPU version
############################

# need to import modules
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
from time import time

# begin timing
start_time = time()

########
R = 10240
C = 10240
numpy.random.seed(123)
a = numpy.random.randn(R, C)
a = a.astype(numpy.float32)
b = numpy.zeros(shape=(R, C), dtype=numpy.uint32)
c = numpy.zeros(shape=(R, C), dtype=numpy.uint32)

# need to allocate memory in the GPU to fit our array
a_gpu = cuda.mem_alloc(a.nbytes)
a_tid = cuda.mem_alloc(b.nbytes)
a_bid = cuda.mem_alloc(c.nbytes)

# need to copy our array to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(a_tid, b)
cuda.memcpy_htod(a_bid, c)

# define the kernel that will run the multiplication in the GPU
mod = SourceModule("""
  __global__ void doublify(float *a, uint *a_tid, uint *a_bid, int R, int C)
  {
    int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
    int idx_y = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = idx_y * C + idx_x;

    if (idx_x < C && idx_y < R) {
        a[idx] *= 2;
        a_tid[idx] = idx_x + blockDim.x * idx_y;
        a_bid[idx] = blockDim.x * blockDim.y;
    }
  }
  """)

func = mod.get_function("doublify")

# define the block and grid dimensions
block = (16, 16, 1)
grid = ((C + block[0] - 1) // block[0], (R + block[1] - 1) // block[1], 1)

# now, we launch the kernel
func(a_gpu, a_tid, a_bid, numpy.int32(R), numpy.int32(C), block=block, grid=grid)

# create space in the host memory to receive results
a_doubled = numpy.empty_like(a)
a_tid_result = numpy.empty_like(b)
a_bid_result = numpy.empty_like(c)

# copy results from the GPU memory to the host memory
cuda.memcpy_dtoh(a_doubled, a_gpu)
cuda.memcpy_dtoh(b, a_tid)
cuda.memcpy_dtoh(c, a_bid)

# end timing
print(round(time() - start_time, 8), 'seconds')

# Optional: Print or verify results
print("tid  bid\n")
for i in range(min(10, len(b))):
    s = ""
    for j in range(min(10, len(b[i]))):
        s += str(b[i, j]) + "  " + str(c[i, j]) + "  "
    print(s)

# Example comparison to verify doubling
print((a * 2 == a_doubled).all())


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


5.03250861 seconds
tid  bid

0  256  1  256  2  256  3  256  4  256  5  256  6  256  7  256  8  256  9  256  
16  256  17  256  18  256  19  256  20  256  21  256  22  256  23  256  24  256  25  256  
32  256  33  256  34  256  35  256  36  256  37  256  38  256  39  256  40  256  41  256  
48  256  49  256  50  256  51  256  52  256  53  256  54  256  55  256  56  256  57  256  
64  256  65  256  66  256  67  256  68  256  69  256  70  256  71  256  72  256  73  256  
80  256  81  256  82  256  83  256  84  256  85  256  86  256  87  256  88  256  89  256  
96  256  97  256  98  256  99  256  100  256  101  256  102  256  103  256  104  256  105  256  
112  256  113  256  114  256  115  256  116  256  117  256  118  256  119  256  120  256  121  256  
128  256  129  256  130  256  131  256  132  256  133  256  134  256  135  256  136  256  137  256  
144  256  145  256  146  256  147  256  148  256  149  256  150  256  151  256  152  256  153  256  
True


# Q1: Have you noticed that the array dimension, the grid shape and the offset calculated by each thread are all related? Increase the dimension of the arrays in both programs and play with the offset and with the `dim3`. Report what you understood about the distribution of threads and blocks and how the threads execute the kernel operations for each program.

Knowing more about the device and the system...

In [13]:
!nvidia-smi

Fri Jun 21 15:54:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0              33W /  70W |   1317MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Collecting device properties using `pycuda`:

In [14]:
import pycuda.driver as drv
import pycuda.autoinit
print("PyCUDA version:", pycuda.VERSION_TEXT)
print("Device(s) found:", drv.Device.count())
for ordinal in range(drv.Device.count()):
  dev = drv.Device(ordinal)
  print("Device number:", ordinal, "Device name:", dev.name())
  print("Compute capability: ", dev.compute_capability())
  print("Max threads per block:", dev.max_threads_per_block)
  print("Max block_dim_x", dev.MAX_BLOCK_DIM_X)
  print("Max block_dim_y", dev.MAX_BLOCK_DIM_Y)
  print("Max block_dim_z", dev.MAX_BLOCK_DIM_Z)
  print("Max grid_dim_x",dev.MAX_GRID_DIM_X)
  print("Max grid_dim_y",dev.MAX_GRID_DIM_Y)
  print("Max grid_dim_z",dev.MAX_GRID_DIM_Z)
  print("Max total constant memory",dev.TOTAL_CONSTANT_MEMORY)
  print("Max warp size",dev.WARP_SIZE)
  print(dev.get_attributes())


PyCUDA version: 2024.1
Device(s) found: 1
Device number: 0 Device name: Tesla T4
Compute capability:  (7, 5)
Max threads per block: 1024
Max block_dim_x 1024
Max block_dim_y 1024
Max block_dim_z 64
Max grid_dim_x 2147483647
Max grid_dim_y 65535
Max grid_dim_z 65535
Max total constant memory 65536
Max warp size 32
{pycuda._driver.device_attribute.ASYNC_ENGINE_COUNT: 3, pycuda._driver.device_attribute.CAN_MAP_HOST_MEMORY: 1, pycuda._driver.device_attribute.CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM: 1, pycuda._driver.device_attribute.CLOCK_RATE: 1590000, pycuda._driver.device_attribute.COMPUTE_CAPABILITY_MAJOR: 7, pycuda._driver.device_attribute.COMPUTE_CAPABILITY_MINOR: 5, pycuda._driver.device_attribute.COMPUTE_MODE: pycuda._driver.compute_mode.DEFAULT, pycuda._driver.device_attribute.COMPUTE_PREEMPTION_SUPPORTED: 1, pycuda._driver.device_attribute.CONCURRENT_KERNELS: 1, pycuda._driver.device_attribute.CONCURRENT_MANAGED_ACCESS: 1, pycuda._driver.device_attribute.DIRECT_MANAGED_MEM_ACCESS

# Q2: inspect the nvidia-smi command and check how you can obtain the same information or more detailed information using the command line.